In [ ]:
from  services.influencers_service import  InfluencersService
from shared.mongo import MongoConnection
mongo_connection=MongoConnection()
influencers_service = InfluencersService(mongo_connection)
bios=[]
posts=influencers_service.get_posts()
influencers=influencers_service.get_influencers()
for influencer in influencers:
    info_string=""
    if influencer['Bio']!= 'NULL':
        info_string+=influencer['Bio']
    if influencer['Category']!= 'NULL':
        info_string+=influencer['Category']
    if info_string!= "":
        bios.append(influencer['Bio']+" "+influencer['Category'])
print(bios[:5])

In [ ]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head()

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def process_data(row):

    text = row['Text']
    text = str(text)
    text = ' '.join(text.split())

    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=128)

    label = 0
    if row['Category'] == 'Food':
        label += 1
    if row['Category'] == 'Beauty':
        label += 2
    
    encodings['label'] = label
    encodings['text'] = text

    return encodings

In [45]:
processed_data = []
for i in range(len(df)):
    processed_data.append(process_data(df.iloc[i]))

In [46]:
from sklearn.model_selection import train_test_split

new_df = pd.DataFrame(processed_data)

train_df, valid_df = train_test_split(
    new_df,
    test_size=0.2,
    random_state=2022
)

In [47]:
import pyarrow as pa
from datasets import Dataset

train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

In [48]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=3
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="./result", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer=tokenizer
)

C:\Users\chadha\PycharmProjects\Influencer-Marketing-2\venv\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [50]:
trainer.train()

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=66, training_loss=0.5234694625392105, metrics={'train_runtime': 276.5215, 'train_samples_per_second': 1.899, 'train_steps_per_second': 0.239, 'total_flos': 34533636076800.0, 'train_loss': 0.5234694625392105, 'epoch': 3.0})

In [51]:
trainer.evaluate()

{'eval_loss': 0.14367276430130005,
 'eval_runtime': 6.2817,
 'eval_samples_per_second': 7.004,
 'eval_steps_per_second': 0.955,
 'epoch': 3.0}

In [52]:
model.save_pretrained('./model/')

In [53]:
from transformers import AutoModelForSequenceClassification
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

new_model = AutoModelForSequenceClassification.from_pretrained('./model/').to(device)

In [54]:
from transformers import AutoTokenizer

new_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [55]:
import torch
import numpy as np

def get_prediction(text):
    encoding = new_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

    outputs = new_model(**encoding)

    logits = outputs.logits
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    sigmoid = torch.nn.Sigmoid()
    print(sigmoid)
    probs = sigmoid(logits.squeeze().cpu())
    probs = probs.detach().numpy()
    label = np.argmax(probs, axis=-1)
    print(logits)
    print(label)

In [1]:
get_prediction('hair | Nails | Fashion Enthusiast YouTuber www.youtube.com/1nOnlyCash')

NameError: name 'get_prediction' is not defined